Hardcode an array saving images and its prediction values

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras.models import Model
from sklearn.utils import shuffle
import savetonparray as savetonp

# import loaddata as loaddata
from google.colab import drive
# %matplotlib inline

# drive.mount('/content/drive')
# !rm -rf '/content/train'
# savetonp.run('train')

In [ ]:
train_x = np.load('train.npy')
test_x = np.load('test.npy')
train_y = pd.read_csv('train_result.csv').iloc[:,1:].values
test_y = pd.read_csv('test_result.csv').iloc[:,1:].values

In [ ]:
train_x = (train_x - np.min(train_x))/(np.max(train_x) - np.min(train_x))
test_x = (test_x - np.min(test_x))/(np.max(test_x) - np.min(test_x))
train_y = train_y/100
test_y = test_y/100

train_x, train_y = shuffle(train_x, train_y)
test_x, test_y = shuffle(test_x, test_y)
# shuffle train labels and samples with respect to each other

print ("train_x shape: " + str(train_x.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x shape: " + str(test_x.shape))
print ("test_y shape: " + str(test_y.shape))
print(train_x[0,:,:,2])
print(train_x[10,:,:,2])
# shuffle*, and add 5 more y result to csv

train_x shape: (335, 299, 299, 15)
train_y shape: (335, 3)
test_x shape: (90, 299, 299, 15)
test_y shape: (90, 3)
[[0.28225806 0.28225806 0.29435484 ... 0.27419355 0.27419355 0.25806452]
 [0.27822581 0.26612903 0.28225806 ... 0.26612903 0.28225806 0.27016129]
 [0.21774194 0.24596774 0.27419355 ... 0.26209677 0.28629032 0.27822581]
 ...
 [0.28225806 0.29032258 0.29435484 ... 0.31048387 0.32258065 0.33467742]
 [0.27822581 0.28629032 0.29032258 ... 0.33064516 0.31854839 0.26612903]
 [0.27419355 0.28629032 0.29032258 ... 0.3266129  0.30241935 0.24596774]]
[[0.26612903 0.25806452 0.25806452 ... 0.29435484 0.23387097 0.22983871]
 [0.26209677 0.26209677 0.27016129 ... 0.26612903 0.22580645 0.24193548]
 [0.26612903 0.25806452 0.25806452 ... 0.27016129 0.25806452 0.27016129]
 ...
 [0.2983871  0.30645161 0.31854839 ... 0.9233871  0.64516129 0.52419355]
 [0.31451613 0.31854839 0.31854839 ... 0.9516129  0.625      0.63709677]
 [0.31048387 0.2983871  0.31048387 ... 0.91935484 0.60080645 0.68548387]

In [ ]:
# build our classifier model based on pre-trained InceptionResNetV2:
# 1. we don't include the top (fully connected) layers of InceptionResNetV2
# 2. we add a DropOut layer followed by a Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')
model = Xception(include_top=True, weights=None, input_shape=(299,299,15), classes=3, classifier_activation="softmax",)

model.compile(optimizer=Adam(learning_rate=1e-5),
                  loss='MSE', metrics=['MAE'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/applications/imagenet_utils.py:333: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 15 input channels.
  str(input_shape[-1]) + ' input channels.')


In [ ]:
print(model.summary())

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 15 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 4320        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [11]:
# train the model
model.fit(train_x, train_y, epochs = 20, batch_size = 10)


Epoch 1/20
34/34 [==============================] - 506s 15s/step - loss: 0.0133 - MAE: 0.0868
Epoch 2/20
34/34 [==============================] - 510s 15s/step - loss: 0.0121 - MAE: 0.0833
Epoch 3/20
34/34 [==============================] - 515s 15s/step - loss: 0.0105 - MAE: 0.0794
Epoch 4/20
34/34 [==============================] - 517s 15s/step - loss: 0.0078 - MAE: 0.0674
Epoch 5/20
34/34 [==============================] - 505s 15s/step - loss: 0.0085 - MAE: 0.0702
Epoch 6/20
34/34 [==============================] - 508s 15s/step - loss: 0.0079 - MAE: 0.0669
Epoch 7/20
34/34 [==============================] - 507s 15s/step - loss: 0.0066 - MAE: 0.0627
Epoch 8/20
34/34 [==============================] - 508s 15s/step - loss: 0.0063 - MAE: 0.0599
Epoch 9/20
34/34 [==============================] - 507s 15s/step - loss: 0.0063 - MAE: 0.0616
Epoch 10/20
34/34 [==============================] - 512s 15s/step - loss: 0.0052 - MAE: 0.0553
Epoch 11/20
34/34 [==============================

In [12]:
# save trained weights
# model.save('15stacked_InceptioResnetV2.h5')
model.save_weights('Xception_20epochs_10batches_Adam_MSE_MAE_normalizedinput.h5')
# predictions = net_final.predict(x=test_batches, verbose=0)

In [13]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
print(preds)

3/3 [==============================] - 38s 11s/step - loss: 0.0154 - MAE: 0.0964
Loss = 0.015412620268762112
Test Accuracy = 0.09642866998910904
[0.015412620268762112, 0.09642866998910904]


In [ ]:
# x_pred1 = np.load('test_img_9.npy')
# print(x_pred1.shape)
# # # img_path = 'resize_5_2.png'
# # img = image.load_img(x, target_size=(224, 224))
# # x = image.img_to_array(img)
# # x = np.expand_dims(x, axis=0)
# x_pred1 = x_pred1/255.0
# print('Input image shape:', x_pred1.shape)
# print("class prediction vector [PR(0), N-PR(1), IM(2)] = ")
# print((net_final.predict(x_pred1)*100).astype(int))
# print(test_y[8]*100)

In [14]:
y_pred_array = np.empty(((test_y.shape[0]), 3), dtype=int)
for i in range (test_x.shape[0]):
  x_pred = test_x[i,:,:,:]
  x_pred = x_pred.reshape((1,299,299,15))
  # print(x_pred.shape)
  y_pred = model.predict(x_pred)
  y_pred_array[i] = (y_pred*100).astype(int)

In [ ]:
print(y_pred_array)
print(test_y*100)
np.savetxt('predicted_y.csv', y_pred_array, fmt='%5.0f', delimiter=",")
np.savetxt('groundtruth_y.csv', test_y*100, fmt='%5.0f', delimiter=",")

[[33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33 32 33]
 [33